In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

import torch as t
import einops
from typing import Dict, List
import torch.sparse as sparse
from tqdm import tqdm
import gc
import torch as t
import torch.sparse as sparse
import einops
from tqdm import tqdm
import pickle
from typing import Dict, List
from pathlib import Path

from trainers.scae import SCAESuite
from buffer import AllActivationBuffer
from utils import load_model_with_folded_ln2, load_iterable_dataset
from find_top_connections import get_top_connections, get_importance_scores

DTYPE = t.bfloat16
device = t.device('cuda' if t.cuda.is_available() else 'cpu')
t.manual_seed(42)
t.set_grad_enabled(False)
gc.collect()

20

In [1]:
model = load_model_with_folded_ln2("gpt2", device=device, torch_dtype=DTYPE)
data = load_iterable_dataset('Skylion007/openwebtext')
suite = SCAESuite.from_pretrained(
    'jacobcd52/gpt2_suite_folded_ln',
    device=device,
    dtype=DTYPE,
    )

initial_submodule = model.transformer.h[0]
layernorm_submodules = {}
submodules = {}
for layer in range(model.config.n_layer):
    submodules[f"mlp_{layer}"] = (model.transformer.h[layer].mlp, "in_and_out")
    submodules[f"attn_{layer}"] = (model.transformer.h[layer].attn, "out")

    layernorm_submodules[f"mlp_{layer}"] = model.transformer.h[layer].ln_2

buffer = AllActivationBuffer(
    data=data,
    model=model,
    submodules=submodules,
    initial_submodule=initial_submodule,
    layernorm_submodules=layernorm_submodules,
    d_submodule=model.config.n_embd,
    n_ctxs=128,
    out_batch_size = 256,
    refresh_batch_size = 256,
    device=device,
    dtype=DTYPE,
)

NameError: name 'load_model_with_folded_ln2' is not defined

In [2]:
# Compute importance scores
importance_scores = get_importance_scores(
    suite, 
    buffer, 
    'mlp_1',
    num_batches=100, 
    save_path=Path('importance_scores.pkl'),
    chunk_size=768
    )

NameError: name 'get_importance_scores' is not defined

In [ ]:
# #%%
# # Later, load importance scores and compute connections
# with open('importance_scores.pkl', 'rb') as f:
#     importance_scores = pickle.load(f)

# connections = get_top_connections(
#     suite=suite,
#     importance_scores=importance_scores,
#     c=10,
#     chunk_size=1000
# )
# # %%